Problema simplificado de los ascensores, primera instancia del problema en el que el edificio contiene un único bloque y todos los ascensores presentan las mismas características.

In [1]:
from search import *
import copy

In [2]:
NUM_PLANTAS    = 5
NUM_PERSONAS   = 3
NUM_ASCENSORES = 1
CAP_ASCENSORES = 2
EN_ASCENSOR = -1

BAJAR=0
SUBIR=1
COGER_PERSONA=2
DEJAR_PERSONA=3

In [3]:
personas_pos_ini = [0]
personas_pos_fin = [1]

In [4]:
class NodeState:
    
    def __init__(self, init_personas):
        self.personas = init_personas 
        self.ascensor_pos = 0
        self.ascensor_personas = []      
    

In [5]:
init = NodeState(personas_pos_ini)

In [6]:
goal = NodeState(personas_pos_fin)

In [7]:
class AscensoresSimplificado(Problem) :    
   
    # Hecho
    def __init__(self, initial, goal=None):        
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        accs = list()
        #Para cada ascensor
        num_ascensor = 0
        ascensor_pos      = state.ascensor_pos
        ascensor_personas = state.ascensor_personas
        personas          = state.personas
        
        #si alguien ha llegado a su destino, hay que forzar que salga
        for persona in ascensor_personas:
            if ascensor_pos==self.goal.personas[persona]:
                accs.append((DEJAR_PERSONA,num_ascensor,persona))
                return accs
        
        # Esto puede ser mejorado, no dejamos una persona que tiene que bajar en un
        # ascensor que baja si no la cambiamos por otra persona que baja!!
        for personaInterior in ascensor_personas:
            accs.append((DEJAR_PERSONA,num_ascensor,personaInterior))
            print("Accion dejar persona "+str(personaInterior))
            
        # Así estamos añadiendo acciones cuya aplicación no modifique el estado
        # Podemos añadir la acción subir cuando no tiene sentido que subamos
        if ascensor_pos != NUM_PLANTAS and not(len(ascensor_personas)>0 and self.goal.personas[ascensor_personas[0]]<ascensor_pos):
            accs.append((SUBIR,num_ascensor))
            print("Accion subir")
        
        # De nuevo estamos añadiendo acciones cuya aplicación puede no modificar el estado
        if ascensor_pos != 0 and not(len(ascensor_personas)>0 and self.goal.personas[ascensor_personas[0]]>ascensor_pos):
            accs.append((BAJAR,num_ascensor))
            print("Accion bajar")
            
        if len(ascensor_personas)<CAP_ASCENSORES:
            for i in range(0,len(personas)):
                if personas[i]==ascensor_pos: #la gente en ascensor tiene pos EN_ASCENSOR
                    accs.append((COGER_PERSONA,num_ascensor,i))
                    print("Accion coger persona "+str(i))
        
        return accs

    def result(self, state, action):
        estado_nuevo = copy.deepcopy(state)
        ascensor          = action[1]
        ascensor_pos      = state.ascensor_pos
        ascensor_personas = state.ascensor_personas
        personas          = state.personas
        
        if action[0]==BAJAR:
            nueva_pos_ascensor = 0 #min bloque
            #si hay gente dentro:
            if(len(ascensor_personas)>0):
                nueva_pos_ascensor = max([self.goal.personas[i] for i in ascensor_personas])
            for i in range(0,len(personas)):
                if nueva_pos_ascensor<personas[i] and personas[i]<ascensor_pos:
                    nueva_pos_ascensor = personas[i]
            estado_nuevo.ascensor_pos = nueva_pos_ascensor
            print("bajar a "+str(nueva_pos_ascensor))
        elif action[0]==SUBIR:
            nueva_pos_ascensor = NUM_PLANTAS #max bloque 
            #si hay gente dentro:
            if(len(ascensor_personas)>0):
                nueva_pos_ascensor = min([self.goal.personas[i] for i in ascensor_personas])
            for i in range(0,len(personas)):
                if nueva_pos_ascensor>personas[i] and personas[i]>ascensor_pos:
                    nueva_pos_ascensor = personas[i]
            estado_nuevo.ascensor_pos = nueva_pos_ascensor
            print("subir a "+str(nueva_pos_ascensor))
        elif action[0]==COGER_PERSONA:
            estado_nuevo.ascensor_personas.append(action[2])
            estado_nuevo.personas[action[2]] = EN_ASCENSOR
            print("coger persona "+str(action[2]))
        else:
            estado_nuevo.ascensor_personas.remove(action[2])
            estado_nuevo.personas[action[2]] = ascensor_pos
            print("dejar persona "+str(action[2]))
        
        return estado_nuevo

    # Hecho
    def goal_test(self, state):
        return state.personas == self.goal.personas

    def path_cost(self, c, state1, action, state2):
        return 1

    def value(self, state):
        raise NotImplementedError

In [8]:
problem = AscensoresSimplificado(init, goal)

In [9]:
breadth_first_graph_search(problem).solution()

Accion subir
Accion coger persona 0
subir a 5
coger persona 0
Accion bajar
bajar a 0
Accion dejar persona 0
Accion subir
dejar persona 0
subir a 1
Accion subir
Accion coger persona 0
subir a 5
coger persona 0
Accion subir
Accion coger persona 0
subir a 5
coger persona 0
dejar persona 0


[(2, 0, 0), (1, 0), (3, 0, 0)]